# Import

In [1]:
import os

import pandas as pd

pd.set_option('display.max_colwidth', -1)

# Latest_Files

In [2]:
directory = os.getenv('NEWS_ITEM_HOME') + 'alileo/'
tableau_data_directory = directory + 'tableau_data/'
files = os.listdir(directory)


def get_datetime_obj_from_file(file_name):
    from datetime import datetime
    name_wo_ext = os.path.splitext(file_name)[0]
    file_name, _date, _time = name_wo_ext.split('_')[:3]
    datetime_obj = datetime.strptime(_date+' '+_time, '%y%m%d %H%M')
    return datetime_obj

def get_latest_files(file_types):
    latest_files = []
    for _type in file_types:
        typed_files = [f for f in files if _type in f]
        file_latest = max(typed_files, key=get_datetime_obj_from_file)
        latest_files.append(file_latest)
    return latest_files
    

file_types = ['alileo1', 'alileo2', 'gochileo1', 'gochileo2']
latest_files = get_latest_files(file_types)
print(latest_files)

['alileo1_190115_1600.json', 'alileo2_190115_1618.json', 'gochileo1_190115_1610.json', 'gochileo2_190116_1125.json']


# Load

In [3]:
df = {}

for file in latest_files:
    name_wo_date = file.split('_')[0]
    df[name_wo_date] = pd.read_json(directory + file, orient='records')
    df[name_wo_date].set_index('id', inplace=True)
    print(name_wo_date)

alileo1
alileo2
gochileo1
gochileo2


# Append

In [4]:
def append_all_df(df_list):
    from functools import reduce
    import pandas as pd
    return reduce(lambda x, y: x.append(y), df_list, pd.DataFrame())

df_all = append_all_df(df.values())
df_all['commentLength'] = df_all['commentText'].str.len()

print(len(df['alileo1']))
print(len(df['alileo2']))
print(len(df['gochileo1']))
print(len(df['gochileo2']))
print(len(df['alileo1']) + len(df['alileo2']) + len(df['gochileo1']) + len(df['gochileo2']))
print(len(df_all))



14917
4395
5731
2490
27533
27533


# Head

In [5]:
# idx_replies_not_na = ~df['alileo1']['replies'].isna()
# idx_replies_more_than_1 = df.reply['numberOfReplies'].astype(int) > 1

# col_replies = ['user', 'numberOfReplies', 'date', 'replies']

df_all.head(1)

,commentText,date,hasReplies,likes,numberOfReplies,replies,timestamp,user,commentLength
id,,,,,,,,,
UgyaPenJFsVpgecX4j94AaABAg,사실 근거 알릴레오~! 유시민 작가님 화이팅입니다 대박나세요,7 minutes ago,False,0,0,NaN,2019-01-15 06:29:00.215,현숙 손,33


# POS

In [6]:
def get_pos_from_comment(df, tagger_name='mecab'):
    tagger_lower = tagger_name.lower()
    if tagger_lower == 'mecab':
        from konlpy.tag import Mecab
        tagger = Mecab()
    elif tagger_lower == 'kkma':
        from konlpy.tag import Kkma
        tagger = Kkma()
    elif tagger_lower == 'okt':
        from konlpy.tag import Okt
        tagger = Okt()
        
    lines = ' '.join(df['commentText'])
    return tagger.pos(lines)


def get_all_tag_set_from_pos(pos):
    return set(tuple(zip(*pos))[1])


def get_n_words_for_tag_from_pos(pos, tag, count=None):
    words = tuple(p[0] for p in pos if p[1]==tag)
    if count:
        return words[:count]
    else:
        return words


def get_counter_from_words(words):
    from collections import Counter
    return Counter(words)


def get_counter_from_df(df, tag, tagger_name='mecab'):
    pos = get_pos_from_comment(df, tagger_name)
    words = get_n_words_for_tag_from_pos(pos, tag)
    return get_counter_from_words(words)


def get_df_from_keyword(df, keyword):
    return df.loc[df['commentText'].str.contains(keyword), ['commentText']]


def get_counter_from_df_by_keyword_and_tag(df, keyword, tag):
    df_keyword = get_df_from_keyword(df, keyword)
    return get_counter_from_df(df_keyword, tag)


def get_suffix_added_counter_from_counter(counter):
    from collections import Counter
    return Counter({k + '다': v for k, v in counter.items()})


def get_suffix_added_counter_from_df(df, tag, tagger_name='mecab'):
    cnt = get_counter_from_df(df, tag, tagger_name)
    return get_suffix_added_counter_from_counter(cnt)


def get_suffix_added_counter_from_df_by_keyword_and_tag(df, keyword, tag):
    cnt = get_counter_from_df_by_keyword_and_tag(df, keyword, tag)
    return get_suffix_added_counter_from_counter(cnt)


def export_df(df, file_path, sep='\t', index=False, reset_index=False):
    if reset_index:
        df.reset_index(inplace=True)
    df.to_csv(file_path, sep=sep, index=index)


def export_counter(counter, file_path, n=10, sep='\t', index=False):
    df_cnt = pd.DataFrame(columns=['key', '횟수'], data=counter.most_common(n))
    export_df(df_cnt, file_path, sep=sep, index=index)

In [53]:
# pos = get_pos_from_comment(df['alileo1'])

print(sum(get_counter_from_df(df_all, 'NNG').values()))
print(get_counter_from_df(df_all, 'NNG'))

186942
Counter({'방송': 2952, '사람': 2610, '대통령': 2505, '감사': 2386, '말': 2255, '국민': 2138, '응원': 2138, '생각': 2081, '정치': 1899, '작가': 1829, '뉴스': 1648, '가짜': 1451, '나라': 1402, '때': 1088, '기대': 981, '핵': 841, '말씀': 813, '문정': 781, '문제': 773, '시민': 747, '이사장': 739, '정부': 736, '앞': 735, '일': 733, '경제': 730, '영상': 723, '보수': 684, '돈': 680, '시간': 678, '평화': 676, '진실': 664, '유작': 660, '트': 645, '가님': 621, '세상': 595, '댓글': 575, '내용': 566, '언론': 558, '사실': 555, '특보': 535, '부탁': 533, '필요': 525, '정권': 511, '국가': 506, '구독자': 505, 'ㄷ': 502, '지금': 491, '홍': 491, '지지': 489, '이해': 487, '사회': 470, '사랑': 470, '구독': 466, '뇌': 465, '존경': 464, '정보': 455, '진보': 449, '행복': 439, '인간': 435, '축하': 434, '시대': 425, '선생': 423, '통일': 418, '이야기': 418, '마음': 418, '정책': 416, '이상': 415, '수준': 411, '끝': 410, '전': 393, '좌파': 388, '다음': 387, '조': 387, '정도': 382, '속': 369, '얘기': 361, '문': 359, '설명': 355, '회수': 353, '광고': 352, '소리': 348, '관계': 348, '힘': 341, '개발': 338, '카': 337, '건강': 335, '콜라': 334, '시작': 328, '중요': 328, '자리'

In [54]:
tmp = get_counter_from_df(df_all, 'NNP')

print(sum(tmp.values()))
print(tmp)

del tmp

30107
Counter({'유시민': 6027, '알릴레오': 2067, '북한': 2018, '레오': 1516, '노무현': 1131, '미국': 793, '화이팅': 683, '대한민국': 659, '유튜브': 634, '문재인': 615, '한국': 503, '김정은': 448, '문정인': 426, '남북': 351, '중국': 336, '유': 336, '브': 283, '북': 275, '유투': 275, '한반도': 269, '노': 248, '일본': 240, '홍준표': 198, '김대중': 183, '민주당': 173, '가즈': 161, '배종찬': 160, '김어준': 160, '박근혜': 127, '레기': 115, '이재명': 111, '빠': 110, '보': 104, '노통': 96, '청와대': 91, '이명박': 90, '유이': 86, '천호선': 84, '추카': 83, '팟캐스트': 81, '정은': 80, '문정부': 80, '파이팅': 78, '노회찬': 74, '조중동': 71, '김': 71, '갈릴레오': 69, '북미': 69, '신잡': 64, '알리': 64, '인도': 63, '네이버': 61, '김정': 59, '유트': 59, '삼성': 56, '김정일': 54, '국당': 54, '남불': 53, '서울': 51, '홍': 51, '주둥': 50, '김일성': 49, '알바': 48, '신재민': 47, '클라스': 46, '서울대': 45, '뷰': 44, '이승만': 42, '노유진': 41, '이명': 40, '페미': 39, '조선': 38, '홍가': 37, '깜': 37, '러시아': 36, '지': 35, '준표': 34, '민주': 33, '평양': 32, '손석희': 32, '유럽': 32, '이사': 32, '예수': 31, '유엔': 29, '박정희': 29, '박': 29, '내요': 28, '최저임금': 28, '베트남': 27, '킹': 27, '문': 26, '적화통일':

In [9]:
print(get_counter_from_df(df_all, 'VV'))

Counter({'하': 6414, '있': 3025, '되': 2777, '보': 2660, '알': 1045, '들': 804, '받': 768, '믿': 754, '듣': 739, '만들': 712, '가': 670, '나오': 606, '살': 530, '맞': 519, '모르': 476, '먹': 421, '넘': 387, '잡': 378, '쓰': 365, '썩': 361, '사': 344, '주': 343, '죽': 300, '보이': 284, '가지': 268, '마시': 241, '아': 238, '남': 232, '그러': 220, '나': 211, '오': 205, '찾': 194, '치': 190, '드리': 186, '올리': 180, '기다리': 177, '만드': 167, '넣': 165, '바라': 158, '찍': 155, '누르': 150, '알리': 148, '풀': 145, '힘내': 128, '깨': 128, '갖': 128, '달': 128, '얻': 127, '뽑': 115, '웃': 114, '빨': 110, '읽': 108, '바꾸': 107, '나가': 104, '드': 103, '속': 103, '지키': 98, '들어가': 98, '배우': 94, '원하': 92, '잃': 90, '말': 89, '물': 89, '느끼': 88, '트': 88, '다루': 85, '놀': 84, '막': 84, '두': 83, '부르': 82, '열': 81, '위하': 81, '들어오': 81, '지': 79, '바뀌': 74, '짖': 74, '빼': 71, '잊': 71, '판치': 71, '차리': 71, '돌아가': 70, '이끌': 69, '내': 69, '생기': 69, '이러': 69, '죽이': 67, '까': 67, '묻': 66, '빠지': 66, '잇': 65, '떨어지': 65, '떠나': 64, '뜨': 62, '뵙': 62, '올라가': 61, '입': 61, '떠들': 59, '타': 58, '더불

In [10]:
print(get_counter_from_df(df_all, 'VA'))

Counter({'좋': 4442, '없': 2760, '같': 2429, '많': 1373, '있': 1345, '싫': 456, '고맙': 395, '쉽': 294, '짧': 274, '다르': 254, '그렇': 239, '반갑': 232, '높': 212, '재밌': 205, '힘들': 180, '길': 169, '젊': 158, '밝': 150, '아쉽': 140, '옳': 137, '재미있': 136, '크': 109, '깊': 94, '그립': 91, '적': 83, '어렵': 83, '멋지': 79, '어떻': 78, '똑같': 76, '안타깝': 76, '늦': 66, '어둡': 61, '괜찮': 60, '아프': 56, '작': 56, '편하': 53, '역겹': 47, '낫': 46, '멀': 42, '즐겁': 41, '나': 40, '빠르': 40, '어리석': 39, '나쁘': 39, '힘드': 39, '멋있': 39, '가볍': 39, '재미없': 38, '넓': 36, '수많': 35, '낮': 34, '무겁': 32, '맑': 31, '무섭': 30, '바쁘': 30, '알차': 30, '이롭': 29, '엄청나': 28, '올바르': 26, '더럽': 26, '기쁘': 25, '아깝': 24, '틀리': 24, '빨': 24, '강하': 24, '심하': 23, '저렇': 22, '어이없': 22, '슬프': 21, '이렇': 21, '상관없': 19, '좁': 19, '외롭': 19, '곱': 18, '새롭': 18, '싸': 18, '부끄럽': 17, '약하': 16, '바르': 15, '쓸데없': 15, '격하': 14, '가깝': 14, '지겹': 14, '굳': 13, '조': 13, '환하': 13, '두렵': 13, '우습': 12, '검': 12, '안쓰럽': 11, '얕': 11, '지나치': 11, '재수없': 11, '짙': 11, '부드럽': 10, '맛있': 10, '급하': 10, '세': 10, '부럽'

In [11]:
tmp_cnt = get_suffix_added_counter_from_counter(get_counter_from_df(df_all, 'VA'))
tmp_file = tableau_data_directory + 'va_counter_general.tsv'

print(tmp_cnt)

# export_counter(tmp_cnt, tmp_file)
del tmp_cnt, tmp_file

Counter({'좋다': 4442, '없다': 2760, '같다': 2429, '많다': 1373, '있다': 1345, '싫다': 456, '고맙다': 395, '쉽다': 294, '짧다': 274, '다르다': 254, '그렇다': 239, '반갑다': 232, '높다': 212, '재밌다': 205, '힘들다': 180, '길다': 169, '젊다': 158, '밝다': 150, '아쉽다': 140, '옳다': 137, '재미있다': 136, '크다': 109, '깊다': 94, '그립다': 91, '적다': 83, '어렵다': 83, '멋지다': 79, '어떻다': 78, '똑같다': 76, '안타깝다': 76, '늦다': 66, '어둡다': 61, '괜찮다': 60, '아프다': 56, '작다': 56, '편하다': 53, '역겹다': 47, '낫다': 46, '멀다': 42, '즐겁다': 41, '나다': 40, '빠르다': 40, '어리석다': 39, '나쁘다': 39, '힘드다': 39, '멋있다': 39, '가볍다': 39, '재미없다': 38, '넓다': 36, '수많다': 35, '낮다': 34, '무겁다': 32, '맑다': 31, '무섭다': 30, '바쁘다': 30, '알차다': 30, '이롭다': 29, '엄청나다': 28, '올바르다': 26, '더럽다': 26, '기쁘다': 25, '아깝다': 24, '틀리다': 24, '빨다': 24, '강하다': 24, '심하다': 23, '저렇다': 22, '어이없다': 22, '슬프다': 21, '이렇다': 21, '상관없다': 19, '좁다': 19, '외롭다': 19, '곱다': 18, '새롭다': 18, '싸다': 18, '부끄럽다': 17, '약하다': 16, '바르다': 15, '쓸데없다': 15, '격하다': 14, '가깝다': 14, '지겹다': 14, '굳다': 13, '조다': 13, '환하다': 13, '두렵다': 13, '우습다': 12, '검다': 12, '안쓰럽다'

In [12]:
tmp_cnt = get_suffix_added_counter_from_df_by_keyword_and_tag(df_all, '방송', 'VA')
tmp_file = tableau_data_directory + 'va_counter_broadcasting.tsv'

print(tmp_cnt)
# export_counter(tmp_cnt, tmp_file)

del tmp_cnt, tmp_file

Counter({'좋다': 975, '같다': 421, '없다': 379, '많다': 229, '있다': 216, '짧다': 63, '재밌다': 55, '고맙다': 54, '쉽다': 53, '높다': 48, '다르다': 43, '싫다': 38, '길다': 38, '재미있다': 36, '그렇다': 35, '밝다': 25, '아쉽다': 24, '크다': 23, '젊다': 21, '힘들다': 21, '깊다': 20, '반갑다': 20, '옳다': 15, '안타깝다': 15, '괜찮다': 15, '나쁘다': 13, '어렵다': 12, '늦다': 12, '적다': 11, '무겁다': 11, '그립다': 10, '어둡다': 9, '작다': 9, '어떻다': 8, '알차다': 8, '재미없다': 8, '똑같다': 8, '수많다': 8, '멋지다': 7, '넓다': 7, '낫다': 7, '어리석다': 7, '즐겁다': 7, '기쁘다': 6, '낮다': 6, '편하다': 6, '새롭다': 6, '아프다': 5, '맑다': 5, '나다': 5, '강하다': 5, '아깝다': 5, '올바르다': 4, '아름답다': 4, '틀리다': 4, '부드럽다': 4, '가볍다': 4, '이렇다': 4, '멀다': 4, '이롭다': 3, '슬프다': 3, '놀랍다': 3, '멋있다': 3, '어이없다': 3, '빠르다': 3, '아무렇다': 3, '좁다': 3, '엄청나다': 3, '검다': 3, '날카롭다': 3, '틀림없다': 3, '오래다': 3, '비싸다': 3, '머다': 3, '힘드다': 3, '무섭다': 3, '잘나다': 3, '격하다': 3, '어지럽다': 2, '뒤늦다': 2, '잘다': 2, '맛있다': 2, '짙다': 2, '그지없다': 2, '더럽다': 2, '지겹다': 2, '매끄럽다': 2, '곱다': 2, '지나치다': 2, '바쁘다': 2, '심하다': 2, '폭넓다': 2, '두껍다': 2, '어떠다': 2, '약하다': 2, '상관없다': 2, '조다': 2,

In [13]:
tmp_cnt = get_suffix_added_counter_from_df_by_keyword_and_tag(df_all, '유시민', 'VA')
tmp_file = tableau_data_directory + 'va_counter_yousimin.tsv'

print(tmp_cnt)
# export_conter(tmp_cnt, tmp_file, n=11)

del tmp_cnt, tmp_file

Counter({'좋다': 1101, '같다': 770, '없다': 728, '있다': 408, '많다': 408, '고맙다': 122, '싫다': 97, '쉽다': 83, '그렇다': 76, '짧다': 69, '힘들다': 67, '젊다': 64, '높다': 54, '반갑다': 54, '재밌다': 50, '옳다': 47, '다르다': 42, '아쉽다': 41, '밝다': 36, '길다': 36, '크다': 34, '깊다': 28, '어떻다': 28, '재미있다': 27, '적다': 27, '멋지다': 25, '안타깝다': 25, '똑같다': 23, '괜찮다': 20, '어렵다': 18, '낫다': 16, '어리석다': 16, '작다': 16, '어둡다': 16, '나다': 15, '늦다': 14, '그립다': 14, '편하다': 14, '무겁다': 14, '나쁘다': 14, '수많다': 13, '아프다': 12, '힘드다': 12, '빠르다': 11, '아깝다': 11, '멀다': 11, '외롭다': 11, '올바르다': 10, '저렇다': 10, '어이없다': 10, '멋있다': 10, '낮다': 9, '가볍다': 9, '강하다': 9, '즐겁다': 9, '기쁘다': 8, '무섭다': 8, '바쁘다': 7, '역겹다': 7, '약하다': 7, '두렵다': 7, '이렇다': 7, '격하다': 7, '더럽다': 7, '알차다': 7, '우습다': 6, '곱다': 6, '틀리다': 6, '부드럽다': 6, '새롭다': 6, '엄청나다': 6, '재미없다': 6, '좁다': 6, '상관없다': 6, '부끄럽다': 5, '가깝다': 5, '맑다': 5, '주제넘다': 5, '싸다': 5, '날카롭다': 4, '안쓰럽다': 4, '속상하다': 4, '감명깊다': 4, '환하다': 4, '쓸데없다': 4, '맛있다': 4, '검다': 4, '바르다': 4, '지겹다': 4, '얇다': 4, '어려우다': 4, '어지럽다': 3, '놀랍다': 3, '슬프다': 3, '잘다

In [14]:
get_df_from_keyword(df_all, '그렇').head()

,commentText
id,
UgxNKY93YDB7Bgt2v1B4AaABAg,"우리나라를 가장 최근에 침략한 국가는 중국과 북한입니다.\n우리나라를 침략한 국가 중에 가장 거대한 병력은 135만명의 중국군입니다. \n중국은 "" 미국만 없었으면 한국은 진작에 손봤을 나라 "" 라는 협박을 하였고\n중국은 "" 소국이 대국에 대항해서 되겠냐 "" 라는 협박을 하였습니다.\n현재 우리나라를 향해 미사일을 조준하고 있는 국가는 중국과 북한입니다.\n반면에 미국은 우리민족을 일본으로 부터 해방 시켜주었습니다.\n8월의 폭풍 작전으로 소련이 한반도를 점령하려했지만\n미국 덕분에 소련군이 38선에서 멈추었고 \n미국 덕분에 38선 아래는 자유민주주의를 누릴 수 있게 되었습니다.\n미국은 자국의 수만명의 청년들을 희생시켜 \n북한군과 중국군으로 부터 우리나라의 자유민주주의를 지켜주었습니다.\n미국은 한국의 GDP 의 2배에 달하는 어마어마한 돈을 원조 해주었고 \n한국이 경제 발전할 수 있도록 미국이 도와주었습니다.\n미국은 한국이 일본으로부터 해방한 직후부터 1970년 5월까지 \n20년 동안 물자와 외화부족 문제를 무상으로 지원해주었습니다.\n특히 1950대말까지는 유일한 외자도입 창구로 \n한국 경제부흥에 미국의 한국 원조 정책이 가장 큰 기여를 해주었습니다.\n미국의 한국 무상원조는 약 44억 달러, 유상원조는 약 4억 달러에 달하였고\n미국의 한국 무상원조는 한국경제의 투자자원 마련, \n국제수지 적자보전 및 경제성장에서 중요한 역할을 했고 \n미국의 원조는 그 당시 가난했던 한국 정부 예산에 큰 도움이 되었습니다.\n1950~70년 한국의 경제는 미국 원조 경제 체제로, \n미국으로부터 농산물, 소비재, 건축자재, 외화 등의 무상 지원을 받았고\n미국의 원조로 식량 문제를 해결하였고 \n1961년 기준으로 한국의 GDP가 23억 달러 수준이었다는 것을 감안하면 \n미국으로부터 받은 무상원조 44억 달러는 그당시 한국 GDP의 2배나 됩니다.\n한국 GDP의 2배나 되는 돈을 무상으로 지원해준 나라가 미국말고 있습니까?\n중국은 지난 5000년 동안 우리의 재산을 약탈하고 공녀를 약탈한 국가입니다.\n언론들은 왜 미국을 대적하고 왜 중국을 찬양 합니까? \n역사를 잊은 민족에게 미래는 없습니다.\n역사를 기억해야 하고 미국에 감사해야합니다.\n미군이 우리나라에 주둔하여 우리나라가 얻는 이득을 돈으로 환산하면 \n그 가치가 1000조원이 넘는다고 합니다.\n미군이 우리나라에 주둔하기 때문에 전세계로부터 투자를 받기 때문입니다\n반면에 미국이 우리나라에 주둔하여 얻는 전략적 이득을 돈으로 환산하면 \n그 가치가 100조원 이하라고 합니다.\n미국은 애치슨 라인만으로도 충분히 세계질서를 유지할 수 있습니다.\n미군 주둔으로 우리나라가 1000조원 이상을 이득보고 있습니다.\n북한에 퍼줄돈 10조원에서 1조원만 줄이면 \n북한에 퍼줄 돈은 9조 원이 되고\n북한에 퍼줄돈 10조원에서 1조원 만 줄이면 \n주한미군 방위비를 1조원에서 2조원으로 올려줄 수 있습니다.\n우리나라를 침략했던 주적 북한에게 9조원만 주고\n우리나라의 경제를 발전시켜주고 우리나라를 잘 살게 해준 \n우리의 은인 미국에게 2조원만 주는 것이 그렇게 아깝습니까?"
UgzSLUEAD5g1V-ewL-54AaABAg,"본인께서 유튜브를 하실거면 고노무현대통령님의 이름이 담긴 `노무현재단` 타이틀 걸지말고 해주십시오. 왜 고노무현대통령님 이름을 등에 엎으려는건데? 양심없으시네요. 고노무현대통령재임당시 당신께서는 그분 돌보기는 커녕 유다같이 행동하지않으셨나요? 당신께서 이야기하신 60대넘은 인재에 대한 본인의 소신을 본인이 어기지마십시오. 당신께서 이야기하신 그 기준에 본인도 어울리지않습니다.... 롤,축구 이야기 안하세요? 왜 게임bj 하시지요? 뭔가 가슴을 채워주지않는 욕심이 마음에 남아맴도나요? 롤 이야기로 지지철회했지만 한때 지지자로서 말씀드립니다. 유시민씨......박수칠때 떠나십시오. 유튜브 아이디어는 혹시 김어준총수의견인가요? 당신때문에 문재인대통령 지지율 대폭 떨어진건 관심없지요? 그 떨어진지지율 회복될거같아? 가짜뉴스청산?좋다그거야. 근데 당신은 아닙니다. 그렇게 말아끼시던분이 왜 말을 그렇게 하셔서 남에게까지피해를 주나요? .................. 앞으로도 안그럴 자신있으세요? 진심?"
UgxJmqtms-lDjH5hKid4AaABAg,"미천하나 한반도 비핵화에 대한 제 생각을 적어봅니다. 현 시점에서 미국은 북한의 핵무기 리스트 공개를 요구하고 북한은 단계적 경제해제를 원하고 있습니다. 하지만 이 부분에서 양국은 교착상태에 빠져있죠. 서로의 불신이 주된 요인이라고 생각합니다. 그렇다면 과연 어떻게 불신을 해소할 수 있을 것인가에 대해 깊이 고민해보아야한다고 생각합니다. 유엔 5개 상임 이사국이 중재자와 같은 보증국이 되어 과거 얄타회담 등과 같은 회담 개최를 제안해야한다고 저는 생각합니다. 의제로는 북한 내 비핵화(한반도 비핵화가 아니라 북한의 비핵화여야 합니다. 한반도 비핵화는 미군의 핵우산 정책에 반하는 것이므로 , 한반도 비핵화는 미국의 지지를 받을 수 없다고 생각합니다.)의 데드라인 설정입니다. 예를들어 연차 단계로 비핵화 프로세스를 미,영,프,러,중 5개 상임 이사국에게 보고 하고, IAEA, UNSC의 사찰을 받아야합니다. 단계적으로 이행시 상임 이사국들이 보증국이 되어 경제제재를 해제하고 지원을 대폭 증가해야합니다. 물론 미 달성시에는 가중제재를 하는 것입니다. \n\n북한 비핵화 문제는 미국 vs 북한의 구도만으로는 풀 수 있는 문제가 아니라고 생각합니다. 하지만 미국이 주출돌 역할을 하는 것을 부정할 수는 없지요. 언제나 힘의 논리에 의해 생각하여만한다고 봅니다. 중국, 프랑스, 독일들 세계 강대국이 한반도 종전선언에 동의하지만, 우리가 그렇게 할 수 없는 것은 트럼프가 시큰둥하기 때문입니다. 이것처럼, 국가 대 국가 사이에 풀기 힘든 문제는 세계 차원으로 끌고 가서 다중 책임제로 해결해야한다고 생각합니다. 이에 유엔 안전 보장 이사회와 상임 이사국이 북한 비핵화 문제에 적극 참여할 수 있도록 우리 정부가 나서야한다고 생각합니다. \n\n*프로그램에 대한 제언입니다. 문정인 특보를 비롯하여 유시민 작가님, 배종찬 본부장님 세 분 모두 훌륭한 말씀을 해주셨습니다. 하지만 말씀으로만 전하시기 보단, PPT와 같이 시청자들이 눈으로 통계 및 자료를 볼 수 있는 도구를 사용하여 말씀을 하신다면, 영상을 보는데 더 좋을 것 같습니다. 그런 통계와 자료를 저희가 보면서 알릴레오의 취지이기도한 국민들을 일깨워주는 효과가 증폭되리라 감히 제언 드립니다. \n\n방송 너무나 좋고 앞으로 애독하겠습니다!"
UgwMl9eFV7kWwSHcfjR4AaABAg,그렇게 미국 싫어하시는 분이 아들은 미국시민권자로 만드셨나요? 앞뒤가 안맞는거같은데....
UgzoqMgKWbDLAruodgZ4AaABAg,"뻔뻔한 얼굴 참 으로 양심없는 놈이구나 유시민! 깡패하고 거래 할때 선금주고 떼이면 누구한테 받을수 있나? 못받는거 다 아는 세상인데, 선금 주고 외상주고 하라구 정은이 깡패한테? 상황논리를 그렇게 하면 믿을 놈이 누굴까? 좌파 종북 쓰레기 놈들만 믿것지!"


# Likes

In [15]:
def get_top_n_column_comment_from_df(df, col='likes', count=1):
    return df.nlargest(count, col)

def get_zero_liked_comment_from_df(df):
    return df.loc[df['likes'] == 0]

In [16]:
# print(type(get_top_n_liked_comment_from_df(df_all)))
# print(get_top_n_liked_comment_from_df(df_all, count=3))

# print(get_top_n_liked_comment_from_df(df_all, col='numberOfReplies', count=5).loc[::, ['commentText', 'timestamp', 'user']])
print(get_top_n_column_comment_from_df(df_all, col='likes', count=10).loc[::, ['commentText', 'date', 'user', 'likes', 'numberOfReplies']])

# print(get_zero_liked_comment_from_df(df_all))

                                                                                                                                                                                                                                                                                                         commentText  \
id                                                                                                                                                                                                                                                                                                                     
UgxyXvSlg4UjkJYDrO54AaABAg  이 글 추천으로 올려주실 수 있나요? 청각장애가 있어 CC자막이라도 누가 달아주실 수 있나요? 영상을 함께 보고 싶어요 ㅠㅠㅠㅠㅠㅠㅠ\n\n💙💙💙💙💙💙💙💙💙자동 자막 보라고 하시는 분들 고맙습니다 \n저도 모르는 사항이 아닙니다ㅠㅠ\n\n자동자막 기능을 알려주신 분들은 먼저 소리 끄고 자동자막을 키고 보시길 권장합니다\n\n자동 자막은 완벽하지 않아서 아예 안보느니만 못해요 ㅠㅠ \n\n그래서 사람이 직접 듣고 쳐주는 자막을 원합니다 ㅠㅠㅠㅠㅠ                                 
Ugxn1dPEhYwbv6jk2fl4AaABAg  알릴레오 보고 판단할레오 ㅎㅎ\n사실에 근거한 합리적 추론의 문화

In [17]:
print((get_counter_from_df(df_all, 'VA')['좋'] / sum(get_counter_from_df(df_all, 'VA').values())))

0.230036250647333


In [18]:
print((get_counter_from_df((get_top_n_column_comment_from_df(df_all, col='likes', count=50)), tag='VA')['좋'] / sum(get_counter_from_df((get_top_n_column_comment_from_df(df_all, col='likes', count=50)), tag='VA').values())))

0.2413793103448276


In [19]:
print(len(get_zero_liked_comment_from_df(df_all)))
print(get_zero_liked_comment_from_df(df_all).tail(20))

14394
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         commentText  \
id                                                                                                                                                        

# User

In [20]:
def get_nlargest_user(df, head=5):
    return df.groupby('user').agg({'commentText': 'count'}).sort_values('commentText', ascending=False).head(head)


def get_user_count(df):
    return df['user'].nunique()


def get_common_user_set_from_df(df_list):
    from functools import reduce
    return reduce(lambda x, y: x & set(y['user'].tolist()), df_list, set(df_list[0]['user'].tolist()))    

In [21]:
# Common User

print(len(get_common_user_set_from_df(tuple(df.values()))))
print(get_common_user_set_from_df(tuple(df.values())))

212
{'이정수', '이루피', '죠엉클', '최동주', '투자여신짱', 'Rak AK', '메리나우', '나상일', '박재철', '선쌤', '최정섭', '김장김치보단겉절이', '슈퍼비타민', '시민구미', 'willis un', '대한국인', '영스토리', '김일혁', '희망별나라', 'stella yan', '저격수', '좋은세상', 'TED Master테드마스터', 'Eunhee Eppinger', 'Ha Nuel Seo', '생강', '안세라', '베드네드', '김인자', '밝은세상', 'hee min', '김석진', '안인우', '더블럽', '남동균', '김정환', 'sunny& puppy', '이병란', 'sunny hwang', 'yeon kim', '황우진', '김비서', '조은정', 'Jaemin Ham', 'Chris Tsui', '김정희', 'Js E', '이상훈', '허병구', 'Saints Kang', 'ᄀ ᄀ', '김아름다운 날들', 'k라니', '홍길동', '강경호', '브라우니', '김민성', '졸린디z z', '크레센뚜평문', '김진석', '짜꼬미', '정은숙', '도카게', 'JuHwan Kim', 'Hyok ki Kwon', 'ohsoony', 'ᄒ ᄒ', 'Kay Kay', 'MH Y', '햄미하모니카', '김주호', '통일은경제다', '미소폭풍', '풀꽃ᆞ', '리차드박', '문순호', '김ustina', 'Cane Caning', '시민', '핱흐', '코스모스', '강주영', 'Young Kim', '오경섭', 'Park Byungsoo', '가을이짱서동욱', 'Shinesoo샤인', '최정연', 'k jgyek', '임현성', '김동욱', '김민경', '김건우', '정예지', '이Anne', 'dong-jin Choi', 'hS kims', '허범정', 'Eun', '이승화', '김정복', '조원화', '박성재', 'lee희정', '김정순', '유로파킴', 'Drone Mania', '상진이', '율사랑', '김순이

In [22]:
# nlargest user

tmp_df = get_nlargest_user(df_all, head=10)
tmp_file = tableau_data_directory + 'nlargest_user.tsv'

print(get_user_count(df_all))
print(tmp_df)

# export_df(tmp_df, tmp_file, reset_index=True)

del tmp_df, tmp_file

# for d in df.alues():
#     print(get_nlargest_user(d, head=5))

18507
          commentText
user                 
김덕기       189        
Jay Kim   131        
스카이       44         
K ND      33         
John kim  27         
레알보수      25         
영민永民      22         
광지영       21         
남정재상      20         
한용민       19         


In [51]:
def get_comment_from_user(df, user, head=5):
    idx_user = df['user']==user
    cols = ['commentText']
    return df.loc[idx_user, cols].head(head)

from functools import partial

head = 5
tmp = partial(get_comment_from_user, head=head)

print(tmp(df_all, '김덕기'))
print(tmp(df_all, 'Jay Kim'))
# print(tmp(df_all, '스카이'))
# print(tmp(df_all, 'K ND'))
# print(tmp(df_all, 'John kim'))
# print(tmp(df_all, '레알보수'))
# print(tmp(df_all, '영민永民'))
# print(tmp(df_all, '광지영'))
# print(tmp(df_all, '남정재상'))
# print(tmp(df_all, '한용민'))

del head, tmp, partial

                                                                                                                                                                                                                                                                                                                                                                             commentText
id                                                                                                                                                                                                                                                                                                                                                                                      
UgzprH-BG1TGpMatDfp4AaABAg  유시민도 안쓰럽다   정부여당이나 진보세력들이 좀 보수 공격에 잘 대응하고 방어하면서 지지율 유질 했으면 유시민이 굳이 위기 상황에 등판하는 일은 없었을텐데 여하튼 어차피 반대진영에서 물고 뜯고 하는 자리에 다시 나온 느낌 본인 성격이 어쩔수 없으니 다시 등판한듯 근데 최대한 외연을 넓히고 문재인 정부의 좋은 소통의 창구가 되었으면 한다                             

# Replies

In [24]:
def get_replies_from_df(df):
    from functools import reduce
    idx_exists = ~df['replies'].isna()
    return reduce(lambda x, y: x + y, df.loc[idx_exists, 'replies'], [])


def get_df_from_list(_list):
    return pd.DataFrame(data=_list).set_index('id')


def get_df_replies_from_df(df):
    replies = get_replies_from_df(df)
    return get_df_from_list(replies)


def get_top_n_replies_from_df(df, n=10):
    return df.nlargest(n, 'numberOfReplies')


def get_mean_number_of_replies_from_df(df):
    return df['numberOfReplies'].mean()


def get_sentences_including_keyword_from_df(df, keyword):
    idx_include_keyword = df['commentText'].str.contains(keyword)
    return df.loc[idx_include_keyword]


def get_number_of_has_replies_from_df(df):
    return df.loc[df['hasReplies'], 'hasReplies'].count()


def get_replies_by_id(df, _id):
    return df.loc[df.index.str.contains(_id)]

In [49]:
df_replies = get_df_replies_from_df(df_all)


from pprint import pprint
# pprint(get_replies_from_df(df_all)[:5])
# print(get_top_n_replies_from_df(df_all)[['commentText', 'numberOfReplies',]])
print(len(df_all))
print(len(df_replies))
print(len(df_all) + len(df_replies))
print('---')
print(len(df['alileo1']))
print(len(df['alileo2']))
print(len(df['gochileo1']))
print(len(df['gochileo2']))

print('---')
print(len(get_df_replies_from_df(df['alileo1'])))
print(len(get_df_replies_from_df(df['alileo2'])))
print(len(get_df_replies_from_df(df['gochileo1'])))
print(len(get_df_replies_from_df(df['gochileo2'])))
print('---')
print(len(df['alileo1']) + len(get_df_replies_from_df(df['alileo1'])))
print(len(df['alileo2']) + len(get_df_replies_from_df(df['alileo2'])))
print(len(df['gochileo1']) + len(get_df_replies_from_df(df['gochileo1'])))
print(len(df['gochileo2']) + len(get_df_replies_from_df(df['gochileo2'])))
print('---')
print(len(df['alileo1']) + len(df['alileo2']) + len(df['gochileo1']) + len(df['gochileo2']) +
      len(get_df_replies_from_df(df['alileo1'])) +
      len(get_df_replies_from_df(df['alileo2'])) +
      len(get_df_replies_from_df(df['gochileo1'])) +
      len(get_df_replies_from_df(df['gochileo2'])))
print(len(df_all))

print(get_mean_number_of_replies_from_df(df_all))

27533
11424
38957
---
14917
4395
5731
2490
---
6189
1456
2130
1649
---
21106
5851
7861
4139
---
38957
27533
0.41492027748519955


## Describe

In [26]:
df_all.describe()

,likes,numberOfReplies,commentLength
count,27533.000000,27533.000000,27533.000000
mean,10.105800,0.414920,61.347619
std,44.216228,2.676841,108.985600
min,0.000000,0.000000,1.000000
25%,0.000000,0.000000,17.000000
50%,0.000000,0.000000,33.000000
75%,2.000000,0.000000,67.000000
max,2235.000000,155.000000,3240.000000


In [27]:
print(get_number_of_has_replies_from_df(df_all))
print(len(df_all))
print(get_number_of_has_replies_from_df(df_all) / len(df_all) * 100)

3118
27533
11.324592307412923


In [58]:
get_replies_by_id(df_replies, 'UgxyXvSlg4UjkJYDrO54AaABAg')

,commentText,date,likes,timestamp,user
id,,,,,
UgxyXvSlg4UjkJYDrO54AaABAg.8phvhbzeEB08phwNf4UPF4,베댓으로 올려줍시다!!! 팟빵게시판에 이 내용 건의하고올게요,1 week ago,90,1546929927958,JM B
UgxyXvSlg4UjkJYDrO54AaABAg.8phvhbzeEB08phyIBZEqiT,좋아요 누르고 갑니다. 자막이 있어야 겠네요,1 week ago,78,1546929927959,박영준
UgxyXvSlg4UjkJYDrO54AaABAg.8phvhbzeEB08phyUPzRXZN,추천드렸어요. 베댓 만들어 드려야 합니다,1 week ago,66,1546929927961,hg song
UgxyXvSlg4UjkJYDrO54AaABAg.8phvhbzeEB08phz9m-IO5t,영상에 커서 대면 하단에 자막 기능 있습니다. 눌러보세요.,1 week ago,33,1546929927962,benzamin Park
UgxyXvSlg4UjkJYDrO54AaABAg.8phvhbzeEB08phzBs-Mun-,"강력 추천합니다.. 알릴레오 제작진 여러분 보편적 알권리를 실현해주세요.. 한글자막을 통해 알권리를 장애를 갖으신 분이나, 비장애인이거나 같아야 합니다.",1 week ago,71,1546929927964,Street Food
UgxyXvSlg4UjkJYDrO54AaABAg.8phvhbzeEB08phzDZYFrB4,화면의 오른쪽 위 터치하시면 점3개가 있어요 그걸 터치하시면 자막보기라고 있어요 쉽게 볼수있어요 ^^~~,1 week ago,32,1546929927965,DNA Vape
UgxyXvSlg4UjkJYDrO54AaABAg.8phvhbzeEB08phzEQ4eTdw,"화면 아랫쪽에 ""자막""이라는 작은 명함같이생긴 아이콘이 있어요 그걸 누르시면 아주 정확하지는 않지만 자막이 나옵니다. 도움 되셨길.",1 week ago,23,1546929927967,jongseon You
UgxyXvSlg4UjkJYDrO54AaABAg.8phvhbzeEB08phzVw7OZxI,빠르게 수정되었으면 좋겠네요!,1 week ago,22,1546929927968,Y K
UgxyXvSlg4UjkJYDrO54AaABAg.8phvhbzeEB08phzh0tVjQu,저도사회적 약자를 존중하는 느낌이 들어 달님을 지지했지만 얼마전 이해찬 대표\n장애인 비하 발언과 김정호 의원의 공항 갑질사건을 보고서 지지를 철회했습니다.,1 week ago,17,1546929927969,박천우


In [29]:
tmp_df = get_top_n_replies_from_df(df_all, n=5)[['commentText', 'user', 'numberOfReplies', 'date', 'likes']]
tmp_file = tableau_data_directory + 'top_5_replies.tsv'

print(tmp_df)
# export_df(tmp_df, tmp_file)

del tmp_df, tmp_file

                                                                                                                                                                                                                                      commentText  \
id                                                                                                                                                                                                                                                  
UgwX9MvPF-2iQogSsLh4AaABAg  많은국민이  고대로  믿고있는현실이  답답합니다   아무리  아니라고 외쳐도  저쪽진영에선  믿지도 듣지도않는다는게  더큰  문제인듯여                                                                                                                                             
UgyMVUO-Mq-oOpcfaOZ4AaABAg  와~~~~~ 꽤 오랜 기간동안 이렇게 수준높고 품격있는 대담을 언제 봤던가 기억이 안난다. \n역시 최곱니다!👏👏👏👏👏👍👍😊\n우파들 이 방송 보고도 남북관계와 통일문제를\n이해하지 못한다면 진짜 호모사피엔스가 아니다.\n이해하고 설득되기 두려워서 듣지 않을지도 모르겠다. 아~~~ 저도 정말 재밌었고 저도 유시민을 백분토론때부터 쫓아 다녔습니다. 두분 정말 존경합니다.👍👍👍👍👍🤩   
Ugx3gTvpGgNRPjdbndt4

In [30]:
print(get_counter_from_df(df_replies, tag='NNP'))

Counter({'북한': 1374, '유시민': 1095, '미국': 620, '노무현': 355, '대한민국': 301, '한국': 266, '김정은': 264, '레오': 263, '문재인': 224, '일본': 213, '박근혜': 201, '중국': 170, '알릴레오': 169, '북': 167, '유튜브': 163, '홍준표': 114, '민주당': 111, '김대중': 105, '이명박': 104, '남북': 94, '이승만': 89, '박정희': 84, '김': 73, '빠': 69, '이명': 68, '노': 66, '박': 66, '유': 65, '유투': 64, '남불': 64, '정청래': 61, '박용진': 60, '브': 59, '일루미나티': 59, '한반도': 53, '페미': 52, '김어준': 52, '전두환': 52, '삼성': 49, '화이팅': 48, '경제성장': 48, '셜': 47, '문정인': 47, '김일성': 47, '러시아': 45, '인도': 43, '가즈': 41, '유엔': 41, '네이버': 39, '조선': 38, '유럽': 37, '노회찬': 35, '문정부': 34, '전라도': 33, '비트코인': 32, '이동과': 32, '최저임금': 31, '영민': 31, '서울': 30, '서울대': 30, '이재명': 30, '청와대': 29, '조중동': 29, '알바': 28, '님': 28, '정은': 26, '홍': 25, '평화통일': 25, '오세웅': 25, '국당': 24, '적화통일': 24, '핵실험': 24, '지': 23, '김정일': 23, '민노총': 23, '킹': 23, '신재민': 22, '소련': 22, '박사모': 22, '노통': 22, '넹': 21, '아베': 20, '트라이엄프': 20, '김정': 19, '독일': 19, '부산': 19, '황장수': 19, '이상실': 19, '브베': 19, '한창석': 19, '박주민': 19, '전교조': 18, '민

In [31]:
print(get_sentences_including_keyword_from_df(df_replies, '북한'))

In [32]:
print(get_sentences_including_keyword_from_df(df_replies, '유시민'))

In [33]:
print(get_counter_from_df(df_replies, tag='SN'))

Counter({'1': 346, '2': 270, '20': 238, '3': 187, '5': 174, '10': 173, '60': 150, '4': 144, '100': 133, '30': 118, '6': 90, '503': 76, '7': 73, '50': 71, '9': 61, '40': 60, '8': 50, '1479': 48, '90': 47, '70': 45, '80': 45, '25': 45, '12': 42, '18': 40, '0': 35, '52': 28, '15': 25, '200': 20, '2011': 15, '16': 14, '625': 14, '13': 14, '11': 14, '21': 14, '640': 14, '99': 14, '110': 14, '1000': 12, '2018': 12, '35': 11, '54': 11, '2002': 11, '59': 10, '2012': 10, '24': 10, '2000': 10, '68': 10, '36': 10, '14': 9, '2016': 9, '2019': 9, '150': 9, '19': 9, '39': 9, '025': 8, '0002873410': 8, '1950': 8, '1947': 8, '17': 8, '00': 8, '123': 8, '2006': 8, '180': 8, '2001': 8, '22': 7, '28': 7, '45': 7, '33': 7, '98': 7, '250': 7, '777': 6, '1952': 6, '38': 6, '29': 6, '2017': 6, '77': 6, '07': 6, '300': 6, '65': 6, '42': 6, '55': 5, '100003642244626': 5, '1429836437147751': 5, '31': 5, '1953': 5, '83': 5, '5000': 5, '2030': 5, '716': 5, '26': 5, '000': 5, '586': 5, '21866213': 5, '95': 4, '518

In [34]:
print(get_sentences_including_keyword_from_df(df_replies, '20').describe())

            likes     timestamp
count  290.000000  2.900000e+02
mean   2.289655    1.547092e+12
std    4.485914    2.516910e+08
min    0.000000    1.546930e+12
25%    0.000000    1.546930e+12
50%    1.000000    1.546930e+12
75%    2.000000    1.547364e+12
max    36.000000   1.547580e+12


In [35]:
print(get_counter_from_df(df_all, tag='SN'))

Counter({'1': 994, '2': 622, '60': 480, '3': 404, '20': 354, '100': 343, '5': 271, '10': 243, '4': 224, '50': 181, '30': 178, '6': 126, '40': 115, '8': 97, '70': 95, '7': 93, '9': 81, '2019': 70, '200': 56, '18': 54, '25': 53, '12': 51, '15': 51, '0': 42, '640': 36, '21': 35, '90': 35, '000': 33, '500': 30, '19': 29, '68': 29, '39': 28, '80': 27, '24': 26, '11': 26, '5000': 23, '16': 23, '518': 22, '300': 22, '1000': 20, '14': 20, '29': 20, '17': 20, '38': 19, '58': 18, '54': 18, '61': 18, '22': 17, '00': 16, '13': 16, '99': 16, '150': 15, '28': 15, '52': 15, '2018': 15, '2000': 14, '23': 13, '51': 13, '33': 12, '27': 12, '55': 12, '625': 12, '2006': 12, '31': 11, '1950': 10, '44': 10, '26': 10, '2030': 10, '56': 9, '34': 9, '45': 9, '2017': 9, '41': 9, '35': 9, '2008': 8, '57': 8, '2020': 8, '2011': 8, '65': 8, '59': 8, '2012': 8, '600': 7, '47': 7, '85': 7, '53': 7, '160': 7, '37': 7, '066': 7, '62': 6, '135': 6, '180': 6, '2001': 6, '2007': 6, '2005': 6, '176': 6, '130': 5, '42': 5,

In [36]:
print(get_sentences_including_keyword_from_df(df_replies, '60')['commentText'])

id
UgxKmrhIlJqIbHw5tBd4AaABAg.8ppmD8nrbgo8ppwiazaexR    60이 되면 가능한 책임있는자리 안가고 좀 더들면 더 안가것다 이게 주제 아니였나 싶은데??  동영상은 보셨는지???                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
UgyOR-HonWGAA285jFh4AaABAg.8ppNNSqKa_g8ppOS3dnwrE    니 꼴통도 썩었어요? 사람 나름이야.. 대한민국에 골통 썩은 자들 많아 60 대 이상 태극기 부

# Longest

In [37]:
def get_longest_n_comment_from_df(df, n=10):
#     idx_sorted_len = df['commentLentgh'].str.len().sort_values(ascending=False).index
#     return df.reindex(idx_sorted_len).head(n)
    return df.sort_values('commentLength', ascending=False).head(n)


def get_character_count_by_user(df):
    return df.groupby('user').agg({'commentLength': 'sum'}).sort_values('commentLength', ascending=False)

In [38]:
print(get_longest_n_comment_from_df(df_all)[['commentText', 'user', 'likes', 'commentLength']])

In [39]:
tmp_df = get_character_count_by_user(df_all).head(10)
tmp_file = tableau_data_directory + 'longest_character_user.tsv'

print(tmp_df)
# export_df(tmp_df, tmp_file, reset_index=True)

del tmp_df, tmp_file

          commentLength
user                   
김덕기       26767        
Jay Kim   14079        
juen      11291        
스카이       5619         
onyx      5408         
가로수       4633         
한용민       4364         
K ND      3375         
sea bull  3352         
ha kim    3063         


In [57]:

# 38957
print(2639754 + 1246027 + 936362 + 461136)
print(38957 / (2639754 + 1246027 + 936362 + 461136) * 100)

5283279
0.7373640498637305
